# Learning and Decision Making

## Laboratory 3: Partially observable Markov decision problems

In the end of the lab, you should export the notebook to a Python script (``File >> Download as >> Python (.py)``). Make sure that the resulting script includes all code written in the tasks marked as "**Activity n. N**", together with any replies to specific questions posed. Your file should be named `padi-labKK-groupXXX.py`, where `KK` corresponds to the lab number and the `XXX` corresponds to your group number. Similarly, your homework should consist of a single pdf file named `padi-hwKK-groupXXX.pdf`. You should create a zip file with the lab and homework files and submit it in Fenix **at most 30 minutes after your lab is over**.

Make sure to strictly respect the specifications in each activity, in terms of the intended inputs, outputs and naming conventions.

In particular, after completing the activities you should be able to replicate the examples provided (although this, in itself, is no guarantee that the activities are correctly completed).

### 1. The POMDP model

Consider the following POMDP problem. It describes the decision process of a police chasing a bandit (like a 1d scotland yard game). The game proceeds in rounds. At each round,

* The police can go either left, right, observe or try to catch the bandit.
* If the bandit is in the same location as the police, the police catches the bandit, gets a cost of zero and a new game starts with the police at position 0 and the bandit at position 2.
* If the bandit is not in the same location as the police, the police receives a penalty/cost.
* At each step (except when the bandit is caught) the bandit has a probability of moving left, right, or staying in the same location.
* The position of the bandit can only be seen when police uses action observe; when used, the police perceives the correct location of the bandit (irrespective of the location of the police). Observations are '0' '1' '2' '3' '4' for bandit detect in location the respective location, or 'e' for empty.


In this lab you will use a POMDP based on the aforementioned domain and investigate how to simulate a partially observable Markov decision problem and track its state. You will also compare different MDP heuristics with the optimal POMDP solution.

**Throughout the lab, unless if stated otherwise, use $\gamma=0.99$.**

$$\diamond$$

In this first activity, you will implement a POMDP model in Python. You will start by loading the POMDP information from a `numpy` binary file, using the `numpy` function `load`. The file contains the list of states, actions, observations, transition probability matrices, observation probability matrices, and cost function.

---

#### Activity 1.        

Write a function named `load_pomdp` that receives, as input, a string corresponding to the name of the file with the POMDP information, and a real number $\gamma$ between $0$ and $1$. The loaded file contains 6 arrays:

* An array `X` that contains all the states in the POMDP, represented as strings. In the police-bandit scenario above, for example, there is a total of 25 states, each describing a stage in the game. The state is represented as x(y) where x is the location of the police and y the location of the bandit
* An array `A` that contains all the actions in the POMDP, also represented as strings. In the  domain above, for example, each action is represented as one of the actions 'left', 'right', 'catch', or 'observe'.
* An array `Z` that contains all the observations in the POMDP, also represented as strings. In the domain above, for example, there is a total of 6 observations.
* An array `P` containing `len(A)` subarrays, each with dimension `len(X)` &times; `len(X)` and  corresponding to the transition probability matrix for one action.
* An array `O` containing `len(A)` subarrays, each with dimension `len(X)` &times; `len(Z)` and  corresponding to the observation probability matrix for one action.
* An array `c` with dimension `len(X)` &times; `len(A)` containing the cost function for the POMDP.

Your function should create the POMDP as a tuple `(X, A, Z, (Pa, a = 0, ..., len(A)), (Oa, a = 0, ..., len(A)), c, g)`, where `X` is a tuple containing the states in the POMDP represented as strings (see above), `A` is a tuple containing the actions in the POMDP represented as strings (see above), `Z` is a tuple containing the observations in the POMDP represented as strings (see above), `P` is a tuple with `len(A)` elements, where `P[a]` is an `np.array` corresponding to the transition probability matrix for action `a`, `O` is a tuple with `len(A)` elements, where `O[a]` is an `np.array` corresponding to the observation probability matrix for action `a`, `c` is an `np.array` corresponding to the cost function for the POMDP, and `g` is a float, corresponding to the discount and provided as the argument $\gamma$ of your function. Your function should return the POMDP tuple.

---

In [1]:
import numpy as np

def load_pomdp(fname, gamma):
    """
    Builds an POMDP model from the provided file.

    :param fname: Name of the file containing the POMDP information
    :type: str
    :param gamma: Discount
    :type: float
    :returns: tuple (tuple, tuple, tuple, tuple, tuple, nd.array, float)
    """

    file = open(fname, 'rb')
    npfile = np.load(file)

    X = npfile["X"]
    A = npfile["A"]
    Z = npfile["Z"]
    P = npfile["P"]
    O = npfile["O"]
    c = npfile["c"]

    file.close()

    return (X, A, Z, P, O, c, gamma)

# -- End: load_pomdp

import numpy.random as rand

M = load_pomdp('linearcatch.npz', 0.99)

rand.seed(42)

# States
print('= State space (%i states) =' % len(M[0]))
print('\nStates:')
for i in range(len(M[0])):
    print(M[0][i])

# Random state
x = rand.randint(len(M[0]))
print('\nRandom state: x =', M[0][x])

# Last state
print('\nLast state:', M[0][-1])

# Actions
print('= Action space (%i actions) =' % len(M[1]))
for i in range(len(M[1])):
    print(M[1][i])

# Random action
a = rand.randint(len(M[1]))
print('\nRandom action: a =', M[1][a])

# Observations
print('= Observation space (%i observations) =' % len(M[2]))
print('\nObservations:')
for i in range(len(M[2])):
    print(M[2][i])

# Random observation
z = rand.randint(len(M[2]))
print('\nRandom observation: z =', M[2][z])

# Last state
print('\nLast observation:', M[2][-1])

# Transition probabilities
print('\n= Transition probabilities =')

for i in range(len(M[1])):
    print('\nTransition probability matrix dimensions (action %s):' % M[1][i], M[3][i].shape)
    print('Dimensions add up for action "%s"?' % M[1][i], np.isclose(np.sum(M[3][i]), len(M[0])))

print('\nState-action pair (%s, %s) transitions to state(s)' % (M[0][x], M[1][a]))
print("s' in", np.array(M[0])[np.where(M[3][a][x, :] > 0)])

# Observation probabilities
print('\n= Observation probabilities =')

for i in range(len(M[1])):
    print('\nObservation probability matrix dimensions (action %s):' % M[1][i], M[4][i].shape)
    print('Dimensions add up for action "%s"?' % M[1][i], np.isclose(np.sum(M[4][i]), len(M[0])))

print('\nState-action pair (%s, %s) yields observation(s)' % (M[0][x], M[1][a]))
print("z in", np.array(M[2])[np.where(M[4][a][x, :] > 0)])

# Cost
print('\n= Costs =')

print('\nCost for the state-action pair (%s, %s):' % (M[0][x], M[1][a]))
print('c(s, a) =', M[5][x, a])

# Discount
print('\n= Discount =')
print('\ngamma =', M[6])

= State space (25 states) =

States:
0(0)
0(1)
0(2)
0(3)
0(4)
1(0)
1(1)
1(2)
1(3)
1(4)
2(0)
2(1)
2(2)
2(3)
2(4)
3(0)
3(1)
3(2)
3(3)
3(4)
4(0)
4(1)
4(2)
4(3)
4(4)

Random state: x = 1(1)

Last state: 4(4)
= Action space (4 actions) =
left
right
observ
catch

Random action: a = catch
= Observation space (6 observations) =

Observations:
0
1
2
3
4
e

Random observation: z = 4

Last observation: e

= Transition probabilities =

Transition probability matrix dimensions (action left): (25, 25)
Dimensions add up for action "left"? True

Transition probability matrix dimensions (action right): (25, 25)
Dimensions add up for action "right"? True

Transition probability matrix dimensions (action observ): (25, 25)
Dimensions add up for action "observ"? True

Transition probability matrix dimensions (action catch): (25, 25)
Dimensions add up for action "catch"? True

State-action pair (1(1), catch) transitions to state(s)
s' in ['0(2)']

= Observation probabilities =

Observation probability matri

We provide below an example of application of the function with the file `pomdp.npz` that you can use as a first "sanity check" for your code. Note that, even fixing the seed, the results you obtain may slightly differ.

```python
import numpy.random as rand

M = load_pomdp('pomdp.npz', 0.99)

rand.seed(42)

# States
print('= State space (%i states) =' % len(M[0]))
print('\nStates:')
for i in range(len(M[0])):
    print(M[0][i])

# Random state
x = rand.randint(len(M[0]))
print('\nRandom state: x =', M[0][x])

# Last state
print('\nLast state:', M[0][-1])

# Actions
print('= Action space (%i actions) =' % len(M[1]))
for i in range(len(M[1])):
    print(M[1][i])

# Random action
a = rand.randint(len(M[1]))
print('\nRandom action: a =', M[1][a])

# Observations
print('= Observation space (%i observations) =' % len(M[2]))
print('\nObservations:')
for i in range(len(M[2])):
    print(M[2][i])

# Random observation
z = rand.randint(len(M[2]))
print('\nRandom observation: z =', M[2][z])

# Last state
print('\nLast observation:', M[2][-1])

# Transition probabilities
print('\n= Transition probabilities =')

for i in range(len(M[1])):
    print('\nTransition probability matrix dimensions (action %s):' % M[1][i], M[3][i].shape)
    print('Dimensions add up for action "%s"?' % M[1][i], np.isclose(np.sum(M[3][i]), len(M[0])))
    
print('\nState-action pair (%s, %s) transitions to state(s)' % (M[0][x], M[1][a]))
print("s' in", np.array(M[0])[np.where(M[3][a][x, :] > 0)])

# Observation probabilities
print('\n= Observation probabilities =')

for i in range(len(M[1])):
    print('\nObservation probability matrix dimensions (action %s):' % M[1][i], M[4][i].shape)
    print('Dimensions add up for action "%s"?' % M[1][i], np.isclose(np.sum(M[4][i]), len(M[0])))
    
print('\nState-action pair (%s, %s) yields observation(s)' % (M[0][x], M[1][a]))
print("z in", np.array(M[2])[np.where(M[4][a][x, :] > 0)])

# Cost
print('\n= Costs =')

print('\nCost for the state-action pair (%s, %s):' % (M[0][x], M[1][a]))
print('c(s, a) =', M[5][x, a])

# Discount
print('\n= Discount =')
print('\ngamma =', M[6])
```

Output:

```
= State space (25 states) =

States:
0(0)
0(1)
0(2)
0(3)
0(4)
1(0)
1(1)
1(2)
1(3)
1(4)
2(0)
2(1)
2(2)
2(3)
2(4)
3(0)
3(1)
3(2)
3(3)
3(4)
4(0)
4(1)
4(2)
4(3)
4(4)

Random state: x = 1(1)

Last state: 4(4)
= Action space (4 actions) =
left
right
observ
catch

Random action: a = catch
= Observation space (6 observations) =

Observations:
0
1
2
3
4
e

Random observation: z = 4

Last observation: e

= Transition probabilities =

Transition probability matrix dimensions (action left): (25, 25)
Dimensions add up for action "left"? True

Transition probability matrix dimensions (action right): (25, 25)
Dimensions add up for action "right"? True

Transition probability matrix dimensions (action observ): (25, 25)
Dimensions add up for action "observ"? True

Transition probability matrix dimensions (action catch): (25, 25)
Dimensions add up for action "catch"? True

State-action pair (1(1), catch) transitions to state(s)
s' in ['0(2)']

= Observation probabilities =

Observation probability matrix dimensions (action left): (25, 6)
Dimensions add up for action "left"? True

Observation probability matrix dimensions (action right): (25, 6)
Dimensions add up for action "right"? True

Observation probability matrix dimensions (action observ): (25, 6)
Dimensions add up for action "observ"? True

Observation probability matrix dimensions (action catch): (25, 6)
Dimensions add up for action "catch"? True

State-action pair (1(1), catch) yields observation(s)
z in ['e']

= Costs =

Cost for the state-action pair (1(1), catch):
c(s, a) = 0.0

= Discount =

gamma = 0.99
```

### 2. Sampling

You are now going to sample random trajectories of your POMDP and observe the impact it has on the corresponding belief.

---

#### Activity 2.

Write a function called `gen_trajectory` that generates a random POMDP trajectory using a uniformly random policy. Your function should receive, as input, a POMDP described as a tuple like that from **Activity 1** and two integers, `x0` and `n` and return a tuple with 3 elements, where:

1. The first element is a `numpy` array corresponding to a sequence of `n + 1` state indices, $x_0,x_1,\ldots,x_n$, visited by the agent when following a uniform policy (i.e., a policy where actions are selected uniformly at random) from state with index `x0`. In other words, you should select $x_1$ from $x_0$ using a random action; then $x_2$ from $x_1$, etc.
2. The second element is a `numpy` array corresponding to the sequence of `n` action indices, $a_0,\ldots,a_{n-1}$, used in the generation of the trajectory in 1.;
3. The third element is a `numpy` array corresponding to the sequence of `n` observation indices, $z_1,\ldots,z_n$, experienced by the agent during the trajectory in 1.

The `numpy` array in 1. should have a shape `(n + 1,)`; the `numpy` arrays from 2. and 3. should have a shape `(n,)`.

**Note:** Your function should work for **any** POMDP specified as above.

---

In [2]:
import numpy.random as rnd

def gen_trajectory(pomdp, x0, steps):
    """
    Generates a random trajectory for the provided POMDP from the provided initial state and with
    the provided number of steps.

    :param pomdp: POMDP description
    :type: tuple
    :param x0: Initial state
    :type: int
    :param steps: Number of steps
    :type: int
    :returns: tuple (nd.array, nd.array, nd.array)
    """

    Ns, Na, Nz = len(pomdp[0]), len(pomdp[1]), len(pomdp[2])
    Pa, O = pomdp[3], pomdp[4]

    states = np.zeros(steps + 1).astype(int)
    actions = np.zeros(steps).astype(int)
    observations = np.zeros(steps).astype(int)

    states[0] = x0
    x = x0

    for i in range(steps):
        a = rand.choice(range(Na))
        actions[i] = a
        x = rand.choice(range(Ns), p=Pa[a][x])
        states[i+1] = x
        z = rand.choice(range(Nz), p=O[a][x])
        observations[i] = z

    return states, actions, observations

# -- End: gen_trajectory

rand.seed(42)

# Number of steps and initial state
steps = 10
x0    = 0 # State I

# Generate trajectory
t = gen_trajectory(M, x0, steps)

# Check shapes
print('Shape of state trajectory:', t[0].shape)
print('Shape of state trajectory:', t[1].shape)
print('Shape of state trajectory:', t[2].shape)

# Print trajectory
for i in range(steps):
    print('\n- Time step %i -' % i)
    print('State:', M[0][t[0][i]], '(state %i)' % t[0][i])
    print('Action selected:', M[1][t[1][i]], '(action %i)' % t[1][i])
    print('Resulting state:', M[0][t[0][i+1]], '(state %i)' % t[0][i+1])
    print('Observation:', M[2][t[2][i]], '(observation %i)' % t[2][i])

Shape of state trajectory: (11,)
Shape of state trajectory: (10,)
Shape of state trajectory: (10,)

- Time step 0 -
State: 0(0) (state 0)
Action selected: observ (action 2)
Resulting state: 0(0) (state 0)
Observation: 0 (observation 0)

- Time step 1 -
State: 0(0) (state 0)
Action selected: catch (action 3)
Resulting state: 0(2) (state 2)
Observation: e (observation 5)

- Time step 2 -
State: 0(2) (state 2)
Action selected: observ (action 2)
Resulting state: 0(2) (state 2)
Observation: 2 (observation 2)

- Time step 3 -
State: 0(2) (state 2)
Action selected: left (action 0)
Resulting state: 4(2) (state 22)
Observation: e (observation 5)

- Time step 4 -
State: 4(2) (state 22)
Action selected: right (action 1)
Resulting state: 0(1) (state 1)
Observation: e (observation 5)

- Time step 5 -
State: 0(1) (state 1)
Action selected: right (action 1)
Resulting state: 1(1) (state 6)
Observation: e (observation 5)

- Time step 6 -
State: 1(1) (state 6)
Action selected: left (action 0)
Resulting 

For example, using the POMDP from **Activity 1** you could obtain the following interaction.

```python
rand.seed(42)

# Number of steps and initial state
steps = 10
x0    = 0 # State I

# Generate trajectory
t = gen_trajectory(M, x0, steps)

# Check shapes
print('Shape of state trajectory:', t[0].shape)
print('Shape of state trajectory:', t[1].shape)
print('Shape of state trajectory:', t[2].shape)

# Print trajectory
for i in range(steps):
    print('\n- Time step %i -' % i)
    print('State:', M[0][t[0][i]], '(state %i)' % t[0][i])
    print('Action selected:', M[1][t[1][i]], '(action %i)' % t[1][i])
    print('Resulting state:', M[0][t[0][i+1]], '(state %i)' % t[0][i+1])
    print('Observation:', M[2][t[2][i]], '(observation %i)' % t[2][i])
```

Output:

```
Shape of state trajectory: (11,)
Shape of state trajectory: (10,)
Shape of state trajectory: (10,)

- Time step 0 -
State: 0(0) (state 0)
Action selected: observ (action 2)
Resulting state: 0(0) (state 0)
Observation: 0 (observation 0)

- Time step 1 -
State: 0(0) (state 0)
Action selected: catch (action 3)
Resulting state: 0(2) (state 2)
Observation: e (observation 5)

- Time step 2 -
State: 0(2) (state 2)
Action selected: observ (action 2)
Resulting state: 0(2) (state 2)
Observation: 2 (observation 2)

- Time step 3 -
State: 0(2) (state 2)
Action selected: left (action 0)
Resulting state: 4(2) (state 22)
Observation: e (observation 5)

- Time step 4 -
State: 4(2) (state 22)
Action selected: right (action 1)
Resulting state: 0(1) (state 1)
Observation: e (observation 5)

- Time step 5 -
State: 0(1) (state 1)
Action selected: right (action 1)
Resulting state: 1(1) (state 6)
Observation: e (observation 5)

- Time step 6 -
State: 1(1) (state 6)
Action selected: left (action 0)
Resulting state: 0(1) (state 1)
Observation: e (observation 5)

- Time step 7 -
State: 0(1) (state 1)
Action selected: left (action 0)
Resulting state: 4(1) (state 21)
Observation: e (observation 5)

- Time step 8 -
State: 4(1) (state 21)
Action selected: observ (action 2)
Resulting state: 4(1) (state 21)
Observation: 1 (observation 1)

- Time step 9 -
State: 4(1) (state 21)
Action selected: catch (action 3)
Resulting state: 4(1) (state 21)
Observation: e (observation 5)
```

You will now write a function that samples a given number of possible belief points for a POMDP. To do that, you will use the function from **Activity 2**.

---

#### Activity 3.

Write a function called `sample_beliefs` that receives, as input, a POMDP described as a tuple like that from **Activity 1** and an integer `n`, and return a tuple with `n + 1` elements **or less**, each corresponding to a possible belief state (represented as a $1\times|\mathcal{X}|$ vector). To do so, your function should

* Generate a trajectory with `n` steps from a random initial state, using the function `gen_trajectory` from **Activity 2**.
* For the generated trajectory, compute the corresponding sequence of beliefs, assuming that the agent does not know its initial state (i.e., the initial belief is the uniform belief, and should also be considered).

Your function should return a tuple with the resulting beliefs, **ignoring duplicate beliefs or beliefs whose distance is smaller than $10^{-3}$.**

**Suggestion:** You may want to define an auxiliary function `belief_update` that receives a POMDP, a belief, an action and an observation and returns the updated belief.

**Note:** Your function should work for **any** POMDP specified as above. To compute the distance between vectors, you may find useful `numpy`'s function `linalg.norm`.


---

In [11]:
def belief_update(mdp, B0, a, z):
    Pa, O = mdp[3], mdp[4]
    prod = B0.dot(Pa[a].dot(np.diag(O[a][:,z])))
    B = prod/prod.sum()

    return B

def sample_beliefs(mdp, n):
    """
    Generates a random sample of belief states for the provided POMDP.

    :param mdp: POMDP description
    :type: tuple
    :param steps: Maximum number of sampled beliefs
    :type: int
    :returns: tuple (n x nd.array)
    """

    Ns = len(mdp[0])
    s0 = rand.choice(range(Ns))
    traject = gen_trajectory(mdp, s0, n)
    B_unnormalized = np.ones(Ns).reshape(1,-1)
    B = B_unnormalized / np.sum(B_unnormalized)
    B_list = [B]

    for i in range(n):
        B = belief_update(mdp, B, traject[1][i], traject[2][i])
        close = False
        for b in B_list:
            if np.linalg.norm(B - b) < 1e-3:
                close = True

        if not close:
            B_list.append(B.reshape(1,-1))

    return tuple(B_list)

# -- sample_belief

rand.seed(42)

# 3 sample beliefs + initial belief
B = sample_beliefs(M, 3)
print('%i beliefs sampled:' % len(B))
for i in range(len(B)):
    print(np.round(B[i], 3))
    print('Belief adds to 1?', np.isclose(B[i].sum(), 1.))

# 100 sample beliefs
B = sample_beliefs(M, 100)
print('%i beliefs sampled.' % len(B))

4 beliefs sampled:
[[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
  0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]]
Belief adds to 1? True
[[0.012 0.032 0.24  0.04  0.036 0.036 0.012 0.032 0.04  0.04  0.04  0.036
  0.012 0.032 0.04  0.04  0.04  0.036 0.012 0.032 0.032 0.04  0.04  0.036
  0.012]]
Belief adds to 1? True
[[0.034 0.019 0.029 0.038 0.04  0.04  0.034 0.019 0.029 0.038 0.038 0.04
  0.034 0.019 0.029 0.029 0.038 0.04  0.034 0.019 0.019 0.049 0.178 0.08
  0.034]]
Belief adds to 1? True
[[0.    0.    0.096 0.    0.    0.    0.    0.063 0.    0.    0.    0.
  0.115 0.    0.    0.    0.    0.132 0.    0.    0.    0.    0.595 0.
  0.   ]]
Belief adds to 1? True
94 beliefs sampled.


For example, using the POMDP from **Activity 1** you could obtain the following interaction.

```python
rand.seed(42)

# 3 sample beliefs + initial belief
B = sample_beliefs(M, 3)
print('%i beliefs sampled:' % len(B))
for i in range(len(B)):
    print(np.round(B[i], 3))
    print('Belief adds to 1?', np.isclose(B[i].sum(), 1.))

# 100 sample beliefs
B = sample_beliefs(M, 100)
print('%i beliefs sampled.' % len(B))
```

Output:

```

rand.seed(42)

# 3 sample beliefs + initial belief
B = sample_beliefs(M, 3)
print('%i beliefs sampled:' % len(B))
for i in range(len(B)):
    print(np.round(B[i], 3))
    print('Belief adds to 1?', np.isclose(B[i].sum(), 1.))

# 100 sample beliefs
B = sample_beliefs(M, 100)
print('%i beliefs sampled.' % len(B))

4 beliefs sampled:
[[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
  0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]]
Belief adds to 1? True
[[0.012 0.032 0.24  0.04  0.036 0.036 0.012 0.032 0.04  0.04  0.04  0.036
  0.012 0.032 0.04  0.04  0.04  0.036 0.012 0.032 0.032 0.04  0.04  0.036
  0.012]]
Belief adds to 1? True
[[0.034 0.019 0.029 0.038 0.04  0.04  0.034 0.019 0.029 0.038 0.038 0.04
  0.034 0.019 0.029 0.029 0.038 0.04  0.034 0.019 0.019 0.049 0.178 0.08
  0.034]]
Belief adds to 1? True
[[0.    0.    0.096 0.    0.    0.    0.    0.063 0.    0.    0.    0.
  0.115 0.    0.    0.    0.    0.132 0.    0.    0.    0.    0.595 0.
  0.   ]]
Belief adds to 1? True
94 beliefs sampled.
```

<font color="blue">**Question 1**: Assume the initial belief is, as implemented above, the uniform belief. **Q1.1** If we were to consider a different policy than the random policy to sample beliefs, should we expect to get a different set of sampled beliefs? **Q1.2** Is our code above able to retrieve all possible beliefs that can be attained under all policies (assume we are able to sample an infinite number of trajectories of infinite-length)? </font>

<font color="blue">**Insert answer:** **Q1.1** Yes, we should expect to get a different set of sampled beliefs if we use a different policy instead of the random policy. The policy determines the actions taken at each step, which directly affects the trajectory of states, actions, and observations. A different policy may lead to different sequences of actions and observations, which in turn influence the belief updates. </font>

<font color="blue">**Insert answer:** **Q1.2** No, the code above is not guaranteed to retrieve all possible beliefs that can be attained under all policies, even with an infinite number of trajectories of infinite length. The code uses a random policy to generate trajectories. While a random policy can explore a wide range of beliefs, it may not systematically explore the entire belief space. Some regions of the belief space may be unreachable or unlikely to be visited under a random policy. Moreover, the code ignores beliefs that are too close (distance <
$10^{-3}$) to previously sampled beliefs. While this avoids redundancy, it also means that beliefs in densely populated regions of the belief space may be underrepresented.</font>

### 3. MDP-based heuristics

In this section you are going to compare different heuristic approaches for POMDPs discussed in class.

---

#### Activity 4

Write a function `solve_mdp` that takes as input a POMDP represented as a tuple like that of **Activity 1** and returns a `numpy` array corresponding to the **optimal $Q$-function for the underlying MDP**. Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note:** Your function should work for **any** POMDP specified as above. Feel free to reuse one of the functions you implemented in Lab 2 (for example, value iteration).

---

In [5]:
def solve_mdp(pomdp):
    """
    Computes the optimal Q-function for the underlying MDP.

    :param pomdp: POMDP description
    :type: tuple
    :returns: nd.array
    """

    X, A, Z, Pa, O, C, gamma = pomdp
    Ns, Na = len(X), len(A)
    J = np.zeros(Ns)
    err = 1
    niter = 0

    Q = np.zeros((Ns, Na))

    while err > 1e-8:

        for a in range(Na):
            s1 = C[:, a, None]
            s2 = gamma * Pa[a].dot(J)
            s3 = np.add(s1.reshape(Ns, 1), s2.reshape(Ns, 1))
            Q[:, a, None] = s3

        Jnew = np.min(Q, axis=1, keepdims=True)

        err = np.linalg.norm(J - Jnew)

        J = Jnew
        niter += 1

    return Q

# -- End: solve_mdp

Q = solve_mdp(M)

x = 6 # State C
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])

x = 3 # State 2C
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])

x = 12 # L
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])


Q-values at state 1(1): [71.743 71.664 71.315 71.025]
Best action at state 1(1): catch

Q-values at state 0(3): [71.664 71.979 71.947 71.91 ]
Best action at state 0(3): left

Q-values at state 2(2): [71.743 71.664 71.315 71.025]
Best action at state 2(2): catch


As an example, you can run the following code on the POMDP from **Activity 1**.

```python
Q = solve_mdp(M)

x = 6 # State C
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])

x = 3 # State 2C
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])

x = 12 # L
print('\nQ-values at state %s:' % M[0][x], np.round(Q[x, :], 3))
print('Best action at state %s:' % M[0][x], M[1][np.argmin(Q[x, :])])
```

Output:

```
Q-values at state 1(1): [71.743 71.664 71.315 71.025]
Best action at state 1(1): catch

Q-values at state 0(3): [71.664 71.979 71.947 71.91 ]
Best action at state 0(3): left

Q-values at state 2(2): [71.743 71.664 71.315 71.025]
Best action at state 2(2): catch
```

---

#### Activity 5

You will now test the different MDP heuristics discussed in class. To that purpose, write down a function that, given a belief vector and the solution for the underlying MDP, computes the action prescribed by each of the three MDP heuristics. In particular, you should write down a function named `get_heuristic_action` that receives, as inputs:

* A belief state represented as a `numpy` array like those of **Activity 3**;
* The optimal $Q$-function for an MDP (computed, for example, using the function `solve_mdp` from **Activity 4**);
* A string that can be either `"mls"`, `"av"`, or `"q-mdp"`;

Your function should return an integer corresponding to the index of the action prescribed by the heuristic indicated by the corresponding string, i.e., the most likely state heuristic for `"mls"`, the action voting heuristic for `"av"`, and the $Q$-MDP heuristic for `"q-mdp"`. *In all heuristics, ties should be broken randomly, i.e., when maximizing/minimizing, you should randomly select between all maximizers/minimizers*.

---

In [7]:
def get_heuristic_action(belief, qfunction, heuristic):
    """
    Computes the action prescribed by the selected MDP heuristic at the provided belief.

    :param belief: belief vector
    :type: nd.array
    :param qfunction: optimal q-function for the underlying MDP
    :type: nd.array
    :param heuristic: selected heuristic
    :type: str
    :returns: int
    """

    Ns, Na = len(belief[0]), len(qfunction[0])

    if heuristic == "mls":
        return np.argmin(qfunction[np.argmax(belief), :])

    elif heuristic == "av":
        votes = np.zeros_like(qfunction[0,:])
        for s in range(Ns):
            votes[np.argmin(qfunction[s, :])] += belief[0][s]
        return np.argmax(votes)

    elif heuristic == 'q-mdp':
        values = np.zeros_like(qfunction[0,:])
        for a in range(Na):
            for s in range(Ns):
                values[a] += belief[0][s]*qfunction[s, a]
        return np.argmin(values)

# -- End: get_heuristic_action

rand.seed(42)

for b in B[:10]:

    if np.all(b > 0):
        print('Belief (approx.) uniform')
    else:
        initial = True

        for i in range(len(M[0])):
            if b[0, i] > 0:
                if initial:
                    initial = False
                    print('Belief: [', M[0][i], ': %.3f' % np.round(b[0, i], 3), end='')
                else:
                    print(',', M[0][i], ': %.3f' % np.round(b[0, i], 3), end='')
        print(']')

    print('MLS action:', M[1][get_heuristic_action(b, Q, 'mls')], end='; ')
    print('AV action:', M[1][get_heuristic_action(b, Q, 'av')], end='; ')
    print('Q-MDP action:', M[1][get_heuristic_action(b, Q, 'q-mdp')])

    print()

Belief (approx.) uniform
MLS action: catch; AV action: left; Q-MDP action: catch

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: catch; AV action: right; Q-MDP action: catch

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: left; AV action: left; Q-MDP action: left

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief: [ 0(2) : 0.625, 1(2) : 0.084, 2(2) : 0.119, 3(2) : 0.078, 4(2) : 0.094]
MLS action: right; AV action: right; Q-MDP action: right

Belief: [ 0(1) : 0.008, 0(2) : 0.059, 0(3) : 0.017, 1(1) : 0.012, 1(2) : 0.083, 1(3) : 0.024, 2(1) : 0.008, 2(2) : 0.054, 2(3) : 0.016, 3(1) : 0.009, 3(2) : 0.066, 3(3) : 0.019, 4(1) : 0.063, 4(2) : 0.438, 4(3) : 0.125]
MLS action: left; AV action: left; Q-MDP action: left

Belief (ap

For example, if you run your function in the examples from **Activity 3** using the $Q$-function from **Activity 4**, you can observe the following interaction.

```python
rand.seed(42)

for b in B[:10]:
    
    if np.all(b > 0):
        print('Belief (approx.) uniform')
    else:
        initial = True

        for i in range(len(M[0])):
            if b[0, i] > 0:
                if initial:
                    initial = False
                    print('Belief: [', M[0][i], ': %.3f' % np.round(b[0, i], 3), end='')
                else:
                    print(',', M[0][i], ': %.3f' % np.round(b[0, i], 3), end='')
        print(']')

    print('MLS action:', M[1][get_heuristic_action(b, Q, 'mls')], end='; ')
    print('AV action:', M[1][get_heuristic_action(b, Q, 'av')], end='; ')
    print('Q-MDP action:', M[1][get_heuristic_action(b, Q, 'q-mdp')])

    print()
```

Output:

```
Belief (approx.) uniform
MLS action: left; AV action: right; Q-MDP action: catch

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: catch; AV action: right; Q-MDP action: catch

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief (approx.) uniform
MLS action: left; AV action: left; Q-MDP action: left

Belief (approx.) uniform
MLS action: right; AV action: right; Q-MDP action: right

Belief: [ 0(2) : 0.625, 1(2) : 0.084, 2(2) : 0.119, 3(2) : 0.078, 4(2) : 0.094]
MLS action: right; AV action: right; Q-MDP action: right

Belief: [ 0(1) : 0.008, 0(2) : 0.059, 0(3) : 0.017, 1(1) : 0.012, 1(2) : 0.083, 1(3) : 0.024, 2(1) : 0.008, 2(2) : 0.054, 2(3) : 0.016, 3(1) : 0.009, 3(2) : 0.066, 3(3) : 0.019, 4(1) : 0.063, 4(2) : 0.438, 4(3) : 0.125]
MLS action: left; AV action: left; Q-MDP action: left

Belief (approx.) uniform
MLS action: left; AV action: left; Q-MDP action: catch
```

---

#### Activity 6

You will now test the different heuristics you implemented in the previous question. To do this, you will implement function `test_heuristic`, which receives as arguments:
* A tuple specifying the POMDP;
* The optimal $Q$-function for an MDP (computed, for example, using the function `solve_mdp` from **Activity 4**);
* A string specifying the action-selection heuristic that can be either `"mls"`, `"av"`, or `"q-mdp"`;
* An integer `n` specifying the legnth of each sampled trajectory;
* An integer `NRUNS` specifying the number of sampled trajectories.

The function should first randomly sample an initial state (with equal probability of sampling any state) and consider the initial belief is uniform over the state space; then, the function should let the agent iteratively interact with the environment while updating its belief and choosing actions according to the specified heuristic. The function should sample `NRUNS` trajectories, compute the discounted cumulative costs for each trajectory, and return the mean discounted cumulative costs obtained over the `NRUNS` sampled trajectories.

In [12]:
def test_heuristic(pomdp, initial_state, qfunction, heuristic, n, NRUNS):
    """
    Simulate NRUNS trajectories of length n using a given action-selection heuristic
      to update the belief and select actions in a POMDP.

    :param pomdp: POMDP description
    :type: tuple
    :param initial_state: initial state
    :type: int
    :param qfunction: optimal q-function for the underlying MDP
    :type: nd.array
    :param heuristic: selected heuristic
    :type: str
    :param n: length of trajectories
    :type: int
    :param NRUNS: number of sampled trajectories
    :type: int

    :returns: float
    """

    X, A, Z, P, O, C, gamma = pomdp
    Ns = len(X)
    Nz = len(Z)

    total_discounted_cost = 0

    for _ in range(NRUNS):
        x = initial_state
        belief = np.ones(Ns) / Ns
        discounted_cost = 0
        discount_factor = 1.0

        for _ in range(n):
            a = get_heuristic_action(belief.reshape(1, -1), qfunction, heuristic)
            x_next = np.random.choice(range(Ns), p=P[a][x])
            z = np.random.choice(range(Nz), p=O[a][x_next])

            belief = belief_update(pomdp, belief, a, z)

            discounted_cost += discount_factor * C[x, a]
            discount_factor *= gamma

            x = x_next

        total_discounted_cost += discounted_cost

    return total_discounted_cost / NRUNS

rand.seed(37)

n = 50
NRUNS = 100

initial_state = 2
print("initial_state:", initial_state)

mls_mean = test_heuristic(M, initial_state, Q, "mls", n, NRUNS)
print("MLS heuristic:", mls_mean)

av_mean = test_heuristic(M, initial_state, Q, "av", n, NRUNS)
print("AV heuristic:", av_mean)

q_mdp_mean = test_heuristic(M, initial_state, Q, "q-mdp", n, NRUNS)
print("Q-MDP heuristic:", q_mdp_mean)

initial_state: 2
MLS heuristic: 35.127502325951255
AV heuristic: 39.49939328624626
Q-MDP heuristic: 35.198849155502415


As an example, you can run the following code.

```python
rand.seed(37)

n = 50
NRUNS = 100

initial_state = 2
print("initial_state:", initial_state)

mls_mean = test_heuristic(M, initial_state, Q, "mls", n, NRUNS)
print("MLS heuristic:", mls_mean)

av_mean = test_heuristic(M, initial_state, Q, "av", n, NRUNS)
print("AV heuristic:", av_mean)

q_mdp_mean = test_heuristic(M, initial_state, Q, "q-mdp", n, NRUNS)
print("Q-MDP heuristic:", q_mdp_mean)
```

Output:

```
initial_state: 2
MLS heuristic: 35.72418905862317
AV heuristic: 39.499393286246324
Q-MDP heuristic: 35.1216686334142
```

<font color="blue">**Question 2:** **Q2.1** Which heuristic(s) appear(s) to perform the best? **Q2.2** Do we have any guarantees on the optimality of these action-selection procedures?
Justify. </font>

<font color='blue'>**Insert answer** **Q2.1** If the goal is to minimize the accumulated cost, the best heuristic will be the one with the lowest return value. In this case, the Q-MDP heuristic. </font>

<font color='blue'>**Insert answer** **Q2.2** No, we do not have guarantees that these heuristics will always yield the optimal policy. Each heuristic provides an approximation based on different principles. Since these heuristics do not explicitly solve the POMDP optimally, they provide practical but suboptimal solutions. </font>

<font color="blue">**Question 2:** **Q2.2** In activity 6 what is the critical point in terms of efficiency?</font>

<font color='blue'>**Insert answer** **Q2.2** The critical efficiency point in Activity 6 is the trade-off between computation time and decision quality. Updating the belief state and selecting actions efficiently is crucial. "MLS" is faster but may be inaccurate, while "AV" and "Q-MDP" require more computation. A high NRUNS improves reliability but increases runtime. Balancing accuracy and speed is key to an efficient implementation.</font>